In [ ]:
print("Hello rumble!")
# PDF-Chatbot – RAG + lokales LLM
# Zellen der Reihe nach ausführen (siehe README).

In [ ]:
import os
import nltk
nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True)

from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate




In [ ]:
# Pfad für den lokalen FAISS-Index (wird beim ersten Lauf erzeugt)
FAISS_INDEX_PATH = "./faiss_index"

In [ ]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)

    documents = loader.load()
    
    return documents

In [ ]:
extr_data =load_pdf("data/")

In [ ]:
#extr_data

In [ ]:
def text_splitter(extr_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(extr_data)
    return texts

In [ ]:
text_chunks = text_splitter(extr_data)
print(len(text_chunks))

In [ ]:
#text_chunks

In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
#embeddings

In [ ]:
query_results = embeddings.embed_query("What is the purpose of this document?")

In [ ]:
#query_results

In [ ]:
# Lokaler Vector-Store mit FAISS (keine Cloud, kein API-Key, keine Migration)
doc_search = FAISS.from_documents(text_chunks, embeddings)
doc_search.save_local(FAISS_INDEX_PATH)

In [ ]:
# Lokales LLM laden (Qwen2.5-Coder GGUF) – braucht: pip install llama-cpp-python
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Streaming-Callback: Antwort wird während der Generierung angezeigt
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path="llm_model/qwen2.5-coder-32b-instruct-q4_k_m.gguf",
    n_ctx=1024,
    max_tokens=512,
    temperature=0,
    repeat_penalty=1.2,
    verbose=False,
    callback_manager=callback_manager,
)

In [ ]:
# RAG-Kette: sucht in deiner PDF, antwortet mit dem LLM
from langchain.chains import RetrievalQA

# Falls doc_search fehlt (z.B. nach Kernel-Neustart): FAISS-Index von Disk laden
try:
    doc_search
except NameError:
    from langchain.vectorstores import FAISS
    try:
        FAISS_INDEX_PATH
    except NameError:
        FAISS_INDEX_PATH = "./faiss_index"
    try:
        embeddings
    except NameError:
        import nltk
        nltk.download("punkt", quiet=True)
        nltk.download("punkt_tab", quiet=True)
        from langchain.embeddings import HuggingFaceEmbeddings
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    doc_search = FAISS.load_local(FAISS_INDEX_PATH, embeddings)
    print("Vector-Store von Disk geladen.")

from langchain.prompts import PromptTemplate

# Prompt: ausführlichere Antwort (mehrere Sätze / kurzer Absatz), nur aus dem Text
QA_PROMPT = PromptTemplate(
    template="""Answer in a short paragraph (4-8 sentences) using ONLY the text below. Be clear and complete. End with a period. Do not invent or repeat. If the answer is not in the text, say: Not in the document.

Text:
{context}

Question: {question}

Answer:""",
    input_variables=["context", "question"],
)

retriever = doc_search.as_retriever(search_kwargs={"k": 2})  # 2 Chunks = weniger Kontext, weniger Verwirrung
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_PROMPT},
)

In [ ]:
# Chat mit dem Bot – PDF-Fragen → RAG; allgemeine Fragen → KI antwortet frei
import io
import sys

NUR_PDF = False
SEP = "─" * 60
# Abstand vom besten PDF-Treffer: wenn > Schwellwert → Frage geht nicht um die PDF → KI antwortet frei
PDF_RELEVANZ_SCHWELLE = 0.95

print("Bot: Hallo! Du kannst mich zur PDF befragen oder ganz allgemein mit mir reden.")
print("     Bei PDF-Fragen zeige ich Abschnitte + Zusammenfassung; sonst antworte ich frei.")
print(f"     Beenden: 'ende' oder 'exit'.\n{SEP}")

while True:
    frage = input("\nDu: ").strip()
    if not frage or frage.lower() in ("ende", "exit", "quit"):
        print(f"\nBot: Bis bald!\n{SEP}")
        break
    print(f"Du: {frage}\n")
    print(SEP)
    try:
        docs_mit_score = doc_search.similarity_search_with_score(frage, k=1)
        zur_pdf = docs_mit_score and docs_mit_score[0][1] <= PDF_RELEVANZ_SCHWELLE

        if NUR_PDF:
            quellen = doc_search.similarity_search(frage, k=4)
            if quellen:
                print("\nRelevante Abschnitte aus der PDF:\n")
                for i, doc in enumerate(quellen):
                    text = doc.page_content.strip()
                    if len(text) > 600:
                        text = text[:597].rsplit(" ", 1)[0] + "..."
                    print(f"  [{i+1}] {text}\n")
            else:
                print("\nBot: Kein passender Abschnitt gefunden.")
        elif zur_pdf:
            _out, _err = io.StringIO(), io.StringIO()
            _so, _se = sys.stdout, sys.stderr
            try:
                sys.stdout, sys.stderr = _out, _err
                ergebnis = qa_chain({"query": frage})
            finally:
                sys.stdout, sys.stderr = _so, _se
            antwort = ergebnis["result"]
            quellen = ergebnis.get("source_documents", [])
            if quellen:
                print("\nRelevante Abschnitte aus der PDF:\n")
                for i, doc in enumerate(quellen[:2]):
                    text = doc.page_content.strip()
                    if len(text) > 450:
                        text = text[:447].rsplit(" ", 1)[0] + "..."
                    print(f"  [{i+1}] {text}\n")
            antwort = antwort.strip().split("\n\n")[0].strip()
            if len(antwort) > 1500:
                last_period = antwort[:1500].rfind(".")
                antwort = (antwort[: last_period + 1] if last_period > 300 else antwort[:1497].rsplit(" ", 1)[0] + "...")
            print(f"{SEP}\n\nZusammenfassung:\n\n   {antwort}\n")
        else:
            prompt = f"""Beantworte die folgende Frage vollständig und ausführlich. Gib eine komplette Antwort von Anfang bis Ende.

Frage: {frage}

Antwort:"""
            _err = io.StringIO()
            _se = sys.stderr
            sys.stderr = _err
            try:
                print("\nAntwort:\n\n   ", end="", flush=True)
                llm(prompt)
                print("\n")
            finally:
                sys.stderr = _se
    except Exception as e:
        print(f"\nBot: Fehler: {e}\n")

Bot: Hallo! Du kannst mich zur PDF befragen oder ganz allgemein mit mir reden.
     Bei PDF-Fragen zeige ich Abschnitte + Zusammenfassung; sonst antworte ich frei.
     Beenden: 'ende' oder 'exit'.
────────────────────────────────────────────────────────────
Du: hallo, sag mir etwas über die PDF Datei

────────────────────────────────────────────────────────────

Antwort:

   